In [ ]:
# !pip install ImageHash

In [ ]:
from pymongo import MongoClient
import os
from pathlib import Path
from PIL import Image
import imagehash
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
imagesmeta = db['imagesmeta']

In [ ]:
images_dir = Path('./images')

In [ ]:
class ImageFile ():
    _attrs = [
        'id',
        'post_id',
        'file_path',
        'filename',
        'ext',
        'size',
        'width',
        'height',
        'ahash',
        'phash',
        'pshash',
        'dhash',
        'whash'
    ]

    def digest (self):
        return {a:str(getattr(self, a)) for a in ImageFile._attrs}

    def __init__ (self, file_path):
        self.file_path = file_path
        self.filename = os.path.basename(file_path)
        self.id, self.ext = os.path.splitext(self.filename)
        self._im = Image.open(file_path)
        self.width, self.height = self._im.size

    @property
    def post_id (self):
        return ('/').join(self.file_path.split('/')[-3:-1]) + '/' + self.id

    @property
    def size (self):
        return os.path.getsize(self.file_path)

    @property
    def ahash (self):
        if not hasattr(self, '_ahash'):
            self._ahash = imagehash.average_hash(self._im)
        return self._ahash

    @property
    def phash (self):
        if not hasattr(self, '_phash'):
            self._phash = imagehash.phash(self._im)
        return self._phash

    @property
    def pshash (self):
        if not hasattr(self, '_pshash'):
            self._pshash = imagehash.phash_simple(self._im)
        return self._pshash

    @property
    def dhash (self):
        if not hasattr(self, '_dhash'):
            self._dhash = imagehash.dhash(self._im)
        return self._dhash

    @property
    def whash (self):
        if not hasattr(self, '_whash'):
            self._whash = imagehash.whash(self._im)
        return self._whash

In [ ]:
path = images_dir/'preview'/'tumblr'

imageFiles = [ImageFile(os.path.join(root, name)) for root, dirs, files in os.walk(path) for name in files]

In [ ]:
num_images = len(imageFiles)

In [ ]:
imageFiles[0].ahash - imageFiles[1].ahash

In [ ]:
adistance = np.ndarray(shape=(len(imageFiles), len(imageFiles)))
for i in tqdm(range(len(imageFiles))):
    for j in range(i+1):
        diff = imageFiles[i].ahash - imageFiles[j].ahash
        adistance[i][j] = diff
        adistance[j][i] = diff

In [ ]:
pdistance = np.ndarray(shape=(len(imageFiles), len(imageFiles)))
for i in tqdm(range(len(imageFiles))):
    for j in range(i+1):
        diff = imageFiles[i].phash - imageFiles[j].phash
        pdistance[i][j] = diff
        pdistance[j][i] = diff

In [ ]:
psdistance = np.ndarray(shape=(len(imageFiles), len(imageFiles)))
for i in tqdm(range(len(imageFiles))):
    for j in range(i+1):
        diff = imageFiles[i].pshash - imageFiles[j].pshash
        psdistance[i][j] = diff
        psdistance[j][i] = diff

In [ ]:
wdistance = np.ndarray(shape=(len(imageFiles), len(imageFiles)))
for i in tqdm(range(len(imageFiles))):
    for j in range(i+1):
        diff = imageFiles[i].whash - imageFiles[j].whash
        wdistance[i][j] = diff
        wdistance[j][i] = diff

In [ ]:
ddistance = np.ndarray(shape=(len(imageFiles), len(imageFiles)))
for i in tqdm(range(len(imageFiles))):
    for j in range(i+1):
        diff = imageFiles[i].dhash - imageFiles[j].dhash
        ddistance[i][j] = diff
        ddistance[j][i] = diff

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})

In [ ]:
ax = sns.heatmap(pdistance, linewidth=0.5, vmax=10)
plt.show()

In [ ]:
ax = sns.heatmap(ddistance, linewidth=0.5, vmax=10)
plt.show()

In [ ]:
ax = sns.heatmap(wdistance, linewidth=0.5, vmax=10)
plt.show()

In [ ]:
ax = sns.heatmap(adistance, linewidth=0.5, vmax=10)
plt.show()

In [ ]:
ax = sns.heatmap(psdistance, linewidth=0.5, vmax=10)
plt.show()

In [ ]:
def func ():
    cnt = 0

    for i in range(num_images):
        for j in range(i):
            if abs(pdistance[i][j]) < 10 and i != j:
                print(cnt, i, j, imageFiles[i].post_id, imageFiles[j].post_id, imageFiles[i].phash, imageFiles[j].phash, pdistance[i][j])
                f = plt.figure()
                f.add_subplot(1, 2, 1)
                plt.imshow(np.asarray(imageFiles[i]._im))
                f.add_subplot(1, 2, 2)
                plt.imshow(np.asarray(imageFiles[j]._im))
                cnt += 1
                if cnt == 24:
                    return
func()